In [1]:
import os
import yaml
from PIL import Image, ImageDraw
from tqdm import tqdm
import shutil

# We define the necessary functions to be used

In [2]:
def convert_yml_to_txt_yolov5(input_path, output_path):
    input_yaml = input_path
    output_txt = output_path

    with open(input_yaml, 'r') as file:
        data = yaml.safe_load(file)

    if 'object' in data['annotation']:
        yolo_labels = []

        for obj in data['annotation']['object']:
            label = 0 if obj['name'] == "person" else 1 if obj['name'] == "wheelchair" else 2 if obj['name'] == "push_wheelchair" else 3 if obj['name'] == "crutches" else 4 if obj['name'] == "walking_frame" else -1
            xmin = float(obj['bndbox']['xmin'])
            ymin = float(obj['bndbox']['ymin'])
            xmax = float(obj['bndbox']['xmax'])
            ymax = float(obj['bndbox']['ymax'])

            width = float(data['annotation']['size']['width'])
            height = float(data['annotation']['size']['height'])

            x_center = (xmin + xmax) / (2 * width)
            y_center = (ymin + ymax) / (2 * height)
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            yolo_labels.append(f"{label} {x_center} {y_center} {w} {h}")

        with open(output_txt, 'w') as file:
            for yolo_label in yolo_labels:
                file.write(yolo_label + '\n')

    else:
        open(output_txt, 'w').close()

def contarLineas(file):
    with open(file,'r') as f:
        lineas = f.readlines()
        nLineas = len(lineas)

    return nLineas

def concatenarTestLabels(concatFiles,outputFile):
    with open(outputFile, 'w') as output:
        # Itera sobre la lista de archivos a concatenar
        for file in concatFiles:
            # Abre cada archivo en modo lectura
            with open(file, 'r') as f:
                # Lee el contenido del archivo y escribe en el archivo de salida
                content = f.read()
                output.write(content)

# View images that will form the test and training set, and concatenate list of test files.

In [3]:
nImages = len(os.listdir("Images_RGB"))
print("Total number of images: ", nImages)
nLabels = len(os.listdir("Annotations_RGB")) + len(os.listdir("Annotations_RGB_TestSet2"))
print("Total number of labels: ", nLabels)

# Lista de nombres de archivos que quieres concatenar
files_to_concatenate = ['ImageSets/TestSet1.txt', 'ImageSets/TestSet2_seq1.txt', 'ImageSets/TestSet2_seq2.txt','ImageSets/TestSet2_seq3.txt','ImageSets/TestSet2_seq4.txt']
output_file = 'ImageSets/TestSet_RGB.txt'
concatenarTestLabels(files_to_concatenate,output_file)
nTest = contarLineas("ImageSets/TestSet_RGB.txt")
nTrain = contarLineas("ImageSets/TrainSet_RGB.txt")


print("Number of train images: ", nTrain)
print("Number of test images: ", nTest)

Total number of images:  17079
Total number of labels:  17079
Number of train images:  10961
Number of test images:  6118


# Creating a folder with all the txt files of the labels

In [4]:
labels=[]
for file in os.listdir("Annotations_RGB"):
    labels.append("Annotations_RGB/" + file)
for file in os.listdir("Annotations_RGB_TestSet2"):
    labels.append("Annotations_RGB_TestSet2/" + file)
print(len(labels))

17079


In [5]:
os.makedirs("Annotations_RGB_txt")
for file in tqdm(labels):
   input = file
   output = "Annotations_RGB_txt/" + file[-29:-4] + ".txt"
   convert_yml_to_txt_yolov5(input,output)

print(len(os.listdir("Annotations_RGB_txt")))

100%|████████████████████████████████████████████████████████████████████████████| 17079/17079 [00:37<00:00, 451.51it/s]

17079


# We make a split in Train/Test in a folder called dataYOLOv5

In [6]:
os.makedirs("dataYOLOv5/train/images")
os.makedirs("dataYOLOv5/train/labels")
os.makedirs("dataYOLOv5/test/images")
os.makedirs("dataYOLOv5/test/labels")


1) We take all the names of the files that are going to go to the test folder

In [7]:
filesTest=[]
with open("ImageSets/TestSet_RGB.txt", 'r') as file:
    lines = [line.strip() for line in file]
    for fileName in lines:
        filesTest.append(fileName)


2. We copy the labels in their corresponding folders

In [8]:
x=[]
for file in tqdm(os.listdir("Annotations_RGB_txt")):
    path = "Annotations_RGB_txt/" + file
    if any(x in path for x in filesTest):
        path_a_copiar = "dataYOLOv5/test/labels/" + file
        shutil.copy(path,path_a_copiar)
        os.remove(path)
        x.append("Test")
    else:
        path_a_copiar = "dataYOLOv5/train/labels/" + file
        shutil.copy(path,path_a_copiar)
        os.remove(path)
        x.append("Train")

100%|███████████████████████████████████████████████████████████████████████████| 17079/17079 [00:11<00:00, 1430.46it/s]


3. We copy the images in their corresponding folders

In [9]:
y=[]
for file in tqdm(os.listdir("Images_RGB")):
    path = "Images_RGB/" + file
    if any(x in path for x in filesTest):
        path_a_copiar = "dataYOLOv5/test/images/" + file
        shutil.copy(path,path_a_copiar)
        os.remove(path)
        y.append("Test")
    else:
        path_a_copiar = "dataYOLOv5/train/images/" + file
        shutil.copy(path,path_a_copiar)
        os.remove(path)
        y.append("Train")

100%|████████████████████████████████████████████████████████████████████████████| 17079/17079 [00:18<00:00, 913.11it/s]


In [10]:
print(len(os.listdir("dataYOLOv5/train/images/")))
print(len(os.listdir("dataYOLOv5/train/labels/")))
print(len(os.listdir("dataYOLOv5/test/images/")))
print(len(os.listdir("dataYOLOv5/test/labels/")))

10961
10961
6118
6118


# Create the .yaml file specific to this dataset.

In [11]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [12]:
%%writetemplate dataYOLOv5/data.yaml
names:
- pedestrian
- wheelchair
- push_wheelchair
- crutches
- walking-frame
nc: 5
test: ../DatasetMobilityAid/dataYOLOv5/test/images
train: ../DatasetMobilityAid/dataYOLOv5/train/images
val: ../DatasetMobilityAid/dataYOLOv5/test/images

# READY!